In [ ]:
import numpy as np
import tifffile
import os
import pprint
from itkwidgets import view

import matplotlib.pyplot as plt

## Set the directories

In [ ]:
dir_name = '/mp/nas2/axue/prettycell_new/'
file_names = os.listdir(dir_name)
file_names = sorted(file_names)
base1 =  [x for x in file_names if 'base01' in x]
base2 =  [x for x in file_names if 'base02' in x]
base3 =  [x for x in file_names if 'base03' in x]
base4 =  [x for x in file_names if 'base04' in x]
base5 =  [x for x in file_names if 'base05' in x]
base6 =  [x for x in file_names if 'base06' in x]
# pprint.pprint(base2)

## [Optional] Visualize all the tiles

In [ ]:
organization = [[1,1],
                [2,1],
                [3,1],
                [3,2],
                [2,2],
                [1,2],
                [1,3],
                [2,3],
                [3,3]]
fig, axs = plt.subplots(3, 3)
for i in range(9):
        ch01 = tifffile.imread(os.path.join(dir_name,base6[i*4]))
        ch02 = tifffile.imread(os.path.join(dir_name,base6[i*4+1]))
        ch03 = tifffile.imread(os.path.join(dir_name,base6[i*4+2]))
        ch04 = tifffile.imread(os.path.join(dir_name,base6[i*4+3]))
        axs[organization[i][0]-1,organization[i][1]-1].imshow(np.max(ch04,axis = 0),vmax = 200)
plt.show()


## Organize according to terasticher format

-  Change the x y z sizes according to your imaging configuration

-  Change the amount of overlap 0.9

-  Change to the channel you need

In [ ]:
organization = [[1,1],
                [2,1],
                [3,1],
                [3,2],
                [2,2],
                [1,2],
                [1,3],
                [2,3],
                [3,3]]

def prepare_data_single(img,x_tile,y_tile):
    z_len,_,_ =  img.shape
    x = int((x_tile-1) * 2048 * 3.4 * 0.9) 
    y = int((y_tile-1) * 2048 * 3.4 * 0.9)
    
    directory = os.path.join(output_dir,"{0:06d}".format(x))
    if os.path.isdir(directory):
        pass
    else:
        os.mkdir(directory)
        
    directory = os.path.join(directory,"{0:06d}_{1:06d}/".format(x,y))
    if os.path.isdir(directory):
        pass
    else:
        os.mkdir(directory)
        
    for z_ind in range(z_len):
        z = 8 * z_ind # 0.2 originall, andrew downsample by 4
        tile = np.squeeze(img[z_ind,:,:])
        tifffile.imsave(directory + "{0:06d}.tif".format(z),tile)

           
output_base = '/mp/nas2/ruihan/40x/input/base6/'

if os.path.isdir(output_base):
    pass
else:
    os.mkdir(output_base)
    
for i in range(3,4):
    print(i)
    img = tifffile.imread(os.path.join(dir_name,base6[i*4]))
    img = img.astype(np.uint16)
    
    output_dir = output_base + '/sum/'
    if os.path.isdir(output_dir):
        pass
    else:
        os.mkdir(output_dir)
    prepare_data_single(img,organization[i][0],organization[i][1])
        
    img = np.squeeze(img[:,:,0])
    img = img.astype(np.uint16)
    output_dir = output_base + '/ch01/'
    if os.path.isdir(output_dir):
        pass
    else:
        os.mkdir(output_dir)
    prepare_data_single(img,organization[i][0],organization[i][1])
    
    img = np.squeeze(img[:,:,1])
    img = img.astype(np.uint16)
    output_dir = output_base + '/ch02/'
    if os.path.isdir(output_dir):
        pass
    else:
        os.mkdir(output_dir)
    prepare_data_single(img,organization[i][0],organization[i][1])   
    
    img = np.squeeze(img[:,:,2])
    img = img.astype(np.uint16)
    output_dir = output_base + '/ch03/'
    if os.path.isdir(output_dir):
        pass
    else:
        os.mkdir(output_dir)
    prepare_data_single(img,organization[i][0],organization[i][1])
    
    img = np.squeeze(img[:,:,0])
    img = img.astype(np.uint16)
    output_dir = output_base + '/ch04/'
    if os.path.isdir(output_dir):
        pass
    else:
        os.mkdir(output_dir)
    prepare_data_single(img,organization[i][0],organization[i][1])

# Register with sum channel

- Set the input dir with all the organized tiles

- Set the output dir

In [ ]:
data_dir = '/mp/nas2/ruihan/40x/input/base6/ch01/'
out_dir = '/mp/nas2/ruihan/40x/output/base6/ch01/'

if os.path.isdir(out_dir):
    pass
else:
    os.mkdir(out_dir)
    
cmd = "/mp/nas2/ruihan/TeraStitcher-portable-1.11.10-Linux/terastitcher\
      --import \
      --volin='{}'\
      --ref1=1 --ref2=2 --ref3=3 \
      --vxl1=0.34 --vxl2=0.34 --vxl3=0.8 \
      --volin_plugin='TiledXY|2Dseries'\
      --sparse_data".format(data_dir)
# os.system(cmd)
print(cmd)


cmd = '/mp/nas2/ruihan/TeraStitcher-portable-1.11.10-Linux/terastitcher\
      --displcompute \
      --projin={}xml_import.xml\
      --sV=100 --sH=100\
      --subvoldim=100'.format(data_dir,1)
# os.system(cmd)
print(cmd)


cmd = '/mp/nas2/ruihan/TeraStitcher-portable-1.11.10-Linux/terastitcher\
      --displproj\
      --projin={}xml_displcomp.xml'.format(data_dir)
# os.system(cmd)
print(cmd)

cmd = '/mp/nas2/ruihan/TeraStitcher-portable-1.11.10-Linux/terastitcher\
      --displthres\
      --projin="{}xml_displproj.xml"\
      --threshold=0.7'.format(data_dir)
# os.system(cmd)
print(cmd)

cmd = '/mp/nas2/ruihan/TeraStitcher-portable-1.11.10-Linux/terastitcher\
      --placetiles\
      --projin="{}xml_displthres.xml"'.format(data_dir)
# os.system(cmd)
print(cmd)
   
cmd = '/mp/nas2/ruihan/TeraStitcher-portable-1.11.10-Linux/terastitcher\
      --merge\
      --projin="{}xml_merging.xml"\
      --volout="{}"\
      --volout_plugin="TiledXY|3Dseries"\
      --libtiff_uncompress\
      --imout_format="tif"\
      --resolutions=013'.format(data_dir,out_dir)
# os.system(cmd)
print(cmd)

## Generate xml file for the other channels

In [ ]:
for base in ['base1','base2','base3','base4','base5','base6']:
    for ch in ['ch01','ch02','ch03','ch04']:
    
        f = open("/mp/nas2/ruihan/40x/input/{}/sum/xml_displcomp.xml".format(base), "r")
        str = f.read()
        f.close()

        old = '/mp/nas2/ruihan/40x/input/{}/sum/'.format(base)
        new = '/mp/nas2/ruihan/40x/input/{}/{}/'.format(base,ch)
        str = str.replace(old,new)

        f = open(r"/mp/nas2/ruihan/40x/input/{}/{}/xml_displcomp.xml".format(base,ch),"w+") 
        f.write(str)
        f.close()

## Register other channels

You can either uncomment the os.system(cmd) after each command, or copy all the printed command to a TMUX session.

The latter is recommended because it takes long and the jupyter notebook kernel may be closed when disconnected. 

In [ ]:
import os

def stitch(data_dir,out_dir):

    if os.path.isdir(out_dir):
        pass
    else:
        os.mkdir(out_dir)
    
    cmd = "/mp/nas2/ruihan/TeraStitcher-portable-1.11.10-Linux/terastitcher\
      --import \
      --volin='{}'\
      --ref1=1 --ref2=2 --ref3=3 \
      --vxl1=0.34 --vxl2=0.34 --vxl3=0.8 \
      --volin_plugin='TiledXY|2Dseries'\
      --sparse_data".format(data_dir)
    print(cmd)

#     cmd = '/mp/nas2/ruihan/TeraStitcher-portable-1.11.10-Linux/terastitcher\
#       --displcompute \
#       --projin={}xml_import.xml\
#       --sV=100 --sH=100\
#       --subvoldim=100'.format(data_dir,1)
#     print(cmd)

    cmd = '/mp/nas2/ruihan/TeraStitcher-portable-1.11.10-Linux/terastitcher\
      --displproj\
      --projin={}xml_displcomp.xml'.format(data_dir)
#     os.system(cmd)
    print(cmd)

    cmd = '/mp/nas2/ruihan/TeraStitcher-portable-1.11.10-Linux/terastitcher\
      --displthres\
      --projin="{}xml_displproj.xml"\
      --threshold=0.7'.format(data_dir)
    # os.system(cmd)
    print(cmd)

    cmd = '/mp/nas2/ruihan/TeraStitcher-portable-1.11.10-Linux/terastitcher\
      --placetiles\
      --projin="{}xml_displthres.xml"'.format(data_dir)
    # os.system(cmd)
    print(cmd)
   
    cmd = '/mp/nas2/ruihan/TeraStitcher-portable-1.11.10-Linux/terastitcher\
      --merge\
      --projin="{}xml_merging.xml"\
      --volout="{}"\
      --volout_plugin="TiledXY|3Dseries"\
      --libtiff_uncompress\
      --imout_format="tif"\
      --resolutions=0'.format(data_dir,out_dir)
    # os.system(cmd)
    print(cmd)
    
for base in ['base1']:#,'base3','base4','base5','base6']:
    for ch in ['ch01','ch02','ch03','ch04']:
        data_dir = '/mp/nas2/ruihan/40x/input/{}/{}/'.format(base,ch)
        out_dir = '/mp/nas2/ruihan/40x/output/{}/{}/'.format(base,ch)
        stitch(data_dir,out_dir)

## Inspect the results

In [ ]:
img = tifffile.imread('/mp/nas2/ruihan/40x/output/base5/sum/RES(5891x5896x319)/000000/000000_00-560/000000_00-560_000000.tif')

In [ ]:
plt.figure(figsize = (20,20))
plt.imshow(np.max(img,axis = 0),vmax = 300)